In [1]:
import json
import random
import os
import pickle
import time
import shutil
import sys
import uuid
from collections import defaultdict
from datetime import timedelta
from glob import glob
from itertools import product
from pyspark.sql import functions as sf
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.storagelevel import StorageLevel
from sklearn.ensemble import IsolationForest
from sklearn.metrics import roc_auc_score

import igraph as ig
import leidenalg as la
import numpy as np
import pandas as pd

import settings as s

assert s.FILE_SIZE == "Large"
assert s.HIGH_ILLICIT == True

os.environ["EXT_DATA_TYPE_FOLDER"] = s.OUTPUT_POSTFIX.lstrip("-")

from common import get_weights
from communities_no_reduction import get_communities_spark
from evaluation import cw_confusion_matrix, cw_recall, cw_f1
from features import generate_features_spark, generate_features_udf_wrapper, SCHEMA_FEAT_UDF, CURRENCY_RATES

%load_ext autoreload
%autoreload 2

In [2]:
start_script = time.time()

In [3]:
if (
    sys.version_info.major,
    sys.version_info.minor,
    sys.version_info.micro,
) != (3, 11, 8):
    raise EnvironmentError(
        "Only runs efficiently on Python 3.11.8 (Tested on: Conda 24.1.2 | Apple M3 Pro)"
    )

In [4]:
SPARK_CONF = [
    ("spark.driver.memory", "32g"),
    ("spark.worker.memory", "32g"),
    ("spark.driver.maxResultSize", "32g"),
    ("spark.driver.bindAddress", "127.0.0.1"),
    ("spark.sql.execution.arrow.pyspark.enabled", "true"),
    ("spark.network.timeout", "600s"),
    ("spark.sql.autoBroadcastJoinThreshold", -1)
]

shutil.rmtree("artifacts", ignore_errors=True)
spark = (
    SparkSession.builder.appName("testing")
    .config(conf=SparkConf().setAll(SPARK_CONF))
    .getOrCreate()
)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/27 10:15:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
location_main = os.path.join("features", os.environ["EXT_DATA_TYPE_FOLDER"], "no-reduction")
# shutil.rmtree(location_main, ignore_errors=True)

location_communities_leiden = f"{location_main}{os.sep}communities_leiden.parquet"

location_features_leiden = f"{location_main}{os.sep}features_leiden.parquet"
location_features_ego = f"{location_main}{os.sep}features_ego.parquet"
location_features_2_hop = f"{location_main}{os.sep}features_2_hop.parquet"
location_features_2_hop_out = f"{location_main}{os.sep}features_2_hop_out.parquet"
location_features_2_hop_in = f"{location_main}{os.sep}features_2_hop_in.parquet"
location_features_2_hop_combined = f"{location_main}{os.sep}features_2_hop_combined.parquet"
location_features_source = f"{location_main}{os.sep}features_source.parquet"
location_features_target = f"{location_main}{os.sep}features_target.parquet"

location_flow_dispense = f"{location_main}{os.sep}location_flow_dispense.parquet"
location_flow_passthrough = f"{location_main}{os.sep}location_flow_passthrough.parquet"
location_flow_sink = f"{location_main}{os.sep}location_flow_sink.parquet"

location_comm_as_source_features = f"{location_main}{os.sep}comm_as_source_features.parquet"
location_comm_as_target_features = f"{location_main}{os.sep}comm_as_target_features.parquet"
location_comm_as_passthrough_features = f"{location_main}{os.sep}comm_as_passthrough_features.parquet"
location_comm_as_passthrough_features_reverse = f"{location_main}{os.sep}comm_as_passthrough_features_reverse.parquet"

location_features_node_level = f"{location_main}{os.sep}features_node_level.parquet"

try:
    os.makedirs(location_main)
except FileExistsError:
    pass

In [6]:
%%time

data = spark.read.parquet(s.STAGED_DATA_LOCATION)

CPU times: user 2.2 ms, sys: 1.53 ms, total: 3.73 ms
Wall time: 979 ms


In [7]:
%%time

flows = pd.read_parquet(s.STAGED_CASES_DATA_LOCATION)
flows.loc[:, "src"] = flows["source"].str.slice(0, 8).tolist()
flows.loc[:, "tgt"] = flows["target"].str.slice(0, 8).tolist()
flows_nodes = set(flows["src"].unique()).union(flows["tgt"].unique())

valid_flows = []
flows_hash = defaultdict(list)
for flow_id, flow in flows.groupby("id"):
    flow_nodes = set(flow["src"].unique()).union(flow["tgt"].unique())
    # Any flow with less than 2 source/target does not qualify as a "flow"
    if len(flow_nodes) > 2:
        flow_graph = ig.Graph.DataFrame(flow.loc[:, ["src", "tgt"]], use_vids=False, directed=True)
        num_components = len(flow_graph.connected_components("weak"))
        if num_components == 1:
            valid_flows.append(flow_id)
            for f_n in flow_nodes:
                flows_hash[f_n].append(flow_id)
flows = flows.loc[flows["id"].isin(valid_flows), :].reset_index(drop=True)
flows_nodes_filtered = set(flows["src"].unique()).union(flows["tgt"].unique())
print(len(valid_flows), len(flows_nodes_filtered))

9664 73599
CPU times: user 10.5 s, sys: 129 ms, total: 10.7 s
Wall time: 10.6 s


In [8]:
nodes_filtered = data.select(
    sf.col("source").alias("x")
).union(data.select(sf.col("target").alias("x"))).distinct().toPandas()["x"].tolist()

assert (len(set(nodes_filtered).intersection(flows_nodes_filtered)) / len(flows_nodes_filtered)) == 1
print(len(nodes_filtered))

2092797


In [9]:
%%time

data = data.withColumn("is_laundering", sf.col("is_laundering").cast("boolean"))
edges = data.groupby(["source", "target"]).agg(
    sf.sum("amount").alias("amount")
).toPandas()
weights = get_weights(edges)
edges_agg = edges.set_index(["source", "target"]).join(
    weights.set_index(["source", "target"]), how="left"
).reset_index()
edges_agg.loc[:, "amount_weighted"] = (
    edges_agg.loc[:, "amount"] * 
    (edges_agg.loc[:, "weight"] / edges_agg.loc[:, "weight"].max())
)

CPU times: user 52.8 s, sys: 1.15 s, total: 53.9 s
Wall time: 1min 12s


In [10]:
# Later on, we will reset the variables (to free up memory), while still keeping these intact
to_keep = %who_ls
to_keep = list(to_keep)

In [11]:
%%time

TOP_N = 50
NUM_HOPS = 5

data_input = spark.createDataFrame(edges_agg)
nodes_source = set(edges_agg["source"].unique())
nodes_target = set(edges_agg["target"].unique())
nodes_passthrough = nodes_source.intersection(nodes_target)

%run generate_flow_features.ipynb

comm_as_source_features.to_parquet(location_comm_as_source_features)
comm_as_target_features.to_parquet(location_comm_as_target_features)
comm_as_passthrough_features.to_parquet(location_comm_as_passthrough_features)
comm_as_passthrough_features_reverse.to_parquet(location_comm_as_passthrough_features_reverse)

del comm_as_source_features
del comm_as_target_features
del comm_as_passthrough_features
del comm_as_passthrough_features_reverse

In [12]:
%%time

print("Constructing Leiden communities")

graph = ig.Graph.DataFrame(edges_agg.loc[:, ["source", "target", "amount_weighted"]], use_vids=False, directed=True)
nodes_mapping = {x.index: x["name"] for x in graph.vs()}
communities_leiden = la.find_partition(
    graph, la.ModularityVertexPartition, n_iterations=100, weights="amount_weighted"
)
communities_leiden = [[nodes_mapping[_] for _ in x] for x in communities_leiden]
communities_leiden = [(str(uuid.uuid4()), set(x)) for x in communities_leiden]
sizes_leiden = [len(x[1]) for x in communities_leiden]

with open(location_communities_leiden, "wb") as fl:
    pickle.dump(communities_leiden, fl)

In [13]:
# with open(location_communities_leiden, "rb") as fl:
#     communities_leiden = pickle.load(fl)

In [37]:
%%time

KEEP_TOP_N = 100

data_agg_weights = get_weights(
    data.groupby(["source", "target"])
    .agg(
        sf.sum("amount").alias("amount")
    ).toPandas()
)

data_agg_weights_rev = data_agg_weights.rename(
    columns={"target": "source", "source": "target"}
).loc[:, ["source", "target", "weight"]]
data_agg_weights_ud = pd.concat([data_agg_weights, data_agg_weights_rev], ignore_index=True)
data_agg_weights_ud = data_agg_weights_ud.groupby(["source", "target"]).agg(weight=("weight", "sum")).reset_index()

data_agg_weights_ud.sort_values("weight", ascending=False, inplace=True)
grouped_ud = data_agg_weights_ud.groupby("source").head(KEEP_TOP_N).reset_index(drop=True)
grouped_ud = grouped_ud.groupby("source").agg(targets=("target", set))

total = grouped_ud.index.nunique()
nodes_neighborhoods = {}
for index, (source, targets) in enumerate(grouped_ud.iterrows()):
    community_candidates = {source}
    for target in targets["targets"]:
        community_candidates |= (grouped_ud.loc[target, "targets"] | {target})
    nodes_neighborhoods[source] = set(community_candidates)
    if not (index % 250_000):
        print(index, total)

del data_agg_weights_rev
del data_agg_weights_ud
del grouped_ud

0 2092797
250000 2092797
500000 2092797
750000 2092797
1000000 2092797
1250000 2092797
1500000 2092797
1750000 2092797
2000000 2092797
CPU times: user 2min 59s, sys: 3.5 s, total: 3min 2s
Wall time: 3min 27s


In [ ]:
%%time

print("Constructing 2-hop communities")

communities_2_hop = get_communities_spark(
    nodes_neighborhoods,
    ig.Graph.DataFrame(edges_agg.loc[:, ["source", "target", "amount_weighted"]], use_vids=False, directed=True), 
    os.cpu_count(), spark, 2, "all", 0.01, "amount_weighted"
)
sizes_2_hop = [len(x[1]) for x in communities_2_hop]

Constructing 2-hop communities


[Stage 29:>                                                       (0 + 12) / 12]

In [ ]:
%%time

ts_min = data.select(sf.min("timestamp").alias("x")).collect()[0]["x"] - timedelta(minutes=1)
data_graph_agg = data.groupby(["source", "target", "source_bank", "target_bank", "source_currency"]).agg(
    sf.count("source").alias("num_transactions"),
    sf.sum("amount").alias("amount"),
    sf.sum("source_amount").alias("source_amount"),
    sf.collect_list(sf.array((sf.col("timestamp") - ts_min).cast("long"), sf.col("amount"))).alias("timestamps_amounts"),
)
data_graph_agg_sdf = data_graph_agg.persist(StorageLevel.DISK_ONLY)
print(data_graph_agg_sdf.count())
data_graph_agg = data_graph_agg_sdf.toPandas()
index = ["source", "target"]
edges_agg.loc[:, index + ["weight"]].set_index(index)
data_graph_agg = data_graph_agg.set_index(index).join(
    edges_agg.loc[:, index + ["weight"]].set_index(index), how="left"
).reset_index()
data_graph_agg.loc[:, "amount_weighted"] = (
    data_graph_agg.loc[:, "amount"] * 
    (data_graph_agg.loc[:, "weight"] / data_graph_agg.loc[:, "weight"].max())
)

In [17]:
graph = ig.Graph.DataFrame(data_graph_agg, use_vids=False, directed=True)

In [18]:
%%time

print("Leiden communitites features creation")

features_leiden = generate_features_spark(communities_leiden, graph, spark)
features_leiden = features_leiden.rename(columns={"key": "key_fake"})
communities_leiden_dict = dict(communities_leiden)
features_leiden.loc[:, "key"] = features_leiden.loc[:, "key_fake"].apply(lambda x: communities_leiden_dict[x])
features_leiden = features_leiden.explode("key")
del features_leiden["key_fake"]
features_leiden.set_index("key").to_parquet(location_features_leiden)

In [19]:
%%time

print("2-hop communitites features creation")

features_2_hop = generate_features_spark(communities_2_hop, graph, spark)
features_2_hop.set_index("key").to_parquet(location_features_2_hop)

In [20]:
del graph

In [21]:
%%time

print("Temporal flows features creation")

edges_totals = data.select("source", "target", "amount").groupby(
    ["source", "target"]
).agg(sf.count("amount").alias("amount")).toPandas()
edges_totals = edges_totals.sort_values("amount", ascending=False).reset_index(drop=True)
left_edges = spark.createDataFrame(edges_totals.groupby("target").head(TOP_N).loc[:, ["source", "target"]])
right_edges = spark.createDataFrame(edges_totals.groupby("source").head(TOP_N).loc[:, ["source", "target"]])

columns = ["source", "target", "timestamp", "amount"]

left = left_edges.select(sf.col("source").alias("src"), sf.col("target").alias("tgt")).join(
    data.select(*columns),
    on=(sf.col("src") == sf.col("source")) & (sf.col("tgt") == sf.col("target")),
    how="left"
).drop("src", "tgt").persist(StorageLevel.DISK_ONLY)
select = []
for column in left.columns:
    select.append(sf.col(column).alias(f"left_{column}"))
left = left.select(*select)
right = right_edges.select(sf.col("source").alias("src"), sf.col("target").alias("tgt")).join(
    data.select(*columns),
    on=(sf.col("src") == sf.col("source")) & (sf.col("tgt") == sf.col("target")),
    how="left"
).drop("src", "tgt").persist(StorageLevel.DISK_ONLY)

print(left.count(), right.count())

flows_temporal = left.join(
    right,
    (left["left_target"] == right["source"]) &
    (left["left_timestamp"] <= right["timestamp"]),
    how="inner"
).groupby(["left_source", "left_target", "source", "target"]).agg(
    sf.sum("left_amount").alias("left_amount"),
    sf.sum("amount").alias("amount"),
).drop("left_target").select(
    sf.col("left_source").alias("dispense"),
    sf.col("source").alias("passthrough"),
    sf.col("target").alias("sink"),
    sf.least("left_amount", "amount").alias("amount"),
)

aggregate = [
    sf.sum("amount").alias("amount_sum"),
    sf.mean("amount").alias("amount_mean"),
    sf.median("amount").alias("amount_median"),
    sf.max("amount").alias("amount_max"),
    sf.stddev("amount").alias("amount_std"),
    sf.countDistinct("dispense").alias("dispense_count"),
    sf.countDistinct("passthrough").alias("passthrough_count"),
    sf.countDistinct("sink").alias("sink_count"),
]
for flow_location, flow_type in [
    (location_flow_dispense, "dispense"), (location_flow_passthrough, "passthrough"), (location_flow_sink, "sink")
]:
    print(flow_type)
    flows_temporal_stats = flows_temporal.groupby(flow_type).agg(*aggregate).toPandas()
    flows_temporal_cyclic_stats = flows_temporal.where(
        (sf.col("dispense") == sf.col("sink"))
    ).groupby(flow_type).agg(*aggregate).toPandas()
    flows_temporal_stats = flows_temporal_stats.set_index(flow_type).join(
        flows_temporal_cyclic_stats.set_index(flow_type),
        how="left", rsuffix="_cycle"
    )
    flows_temporal_stats.index.name = "key"
    flows_temporal_stats.to_parquet(flow_location)
    del flows_temporal_stats
    del flows_temporal_cyclic_stats

left.unpersist()
right.unpersist()

del edges_totals
del left_edges
del right_edges

In [22]:
%%time

print("1-hop-source features creation")

features_source = spark.createDataFrame(data_graph_agg).withColumn(
    "key", sf.col("source")
).repartition(os.cpu_count() * 5, "key").groupby("key").applyInPandas(
    generate_features_udf_wrapper(False), schema=SCHEMA_FEAT_UDF
).toPandas()
features_source = pd.DataFrame(features_source["features"].apply(json.loads).tolist())
features_source.columns = [f"{s.G_1HOP_PREFIX}{x}" if x != "key" else x for x in features_source.columns]
features_source.set_index("key").to_parquet(location_features_source)

1-hop-source features creation


25/07/27 10:22:53 WARN TaskSetManager: Stage 20 contains a task of very large size (8990 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

CPU times: user 3min 19s, sys: 21.9 s, total: 3min 41s
Wall time: 33min 51s


In [23]:
%%time

print("1-hop-target features creation")

features_target = spark.createDataFrame(data_graph_agg).withColumn(
    "key", sf.col("target")
).repartition(os.cpu_count() * 5, "key").groupby("key").applyInPandas(
    generate_features_udf_wrapper(False), schema=SCHEMA_FEAT_UDF
).toPandas()
features_target = pd.DataFrame(features_target["features"].apply(json.loads).tolist())
features_target.columns = [f"{s.G_1HOP_PREFIX}{x}" if x != "key" else x for x in features_target.columns]
features_target.set_index("key").to_parquet(location_features_target)

1-hop-target features creation


25/07/27 10:57:32 WARN TaskSetManager: Stage 23 contains a task of very large size (8990 KiB). The maximum recommended task size is 1000 KiB.
                                                                                

CPU times: user 3min 24s, sys: 24 s, total: 3min 48s
Wall time: 25min 10s


In [24]:
del data_graph_agg

In [25]:
ENABLED_FEATURES = [
    ("leiden", location_features_leiden),
    ("2_hop", location_features_2_hop),
    ("as_source", location_features_source),
    ("as_target", location_features_target),
    ("comm_as_source_features", location_comm_as_source_features),
    ("comm_as_target_features", location_comm_as_target_features),
    ("comm_as_passthrough_features", location_comm_as_passthrough_features),
    ("comm_as_passthrough_features_reverse", location_comm_as_passthrough_features_reverse),
    ("flow_dispense", location_flow_dispense),
    ("flow_passthrough", location_flow_passthrough),
    ("flow_sink", location_flow_sink),
]

In [26]:
all_features = pd.DataFrame()
all_features.index.name = "key"

for feature_group, location in ENABLED_FEATURES:
    all_features = all_features.join(
        pd.read_parquet(location), how="outer", rsuffix=f"_{feature_group}"
    )

all_features.to_parquet(location_features_node_level)
print("Features:", all_features.shape)
del all_features

Features: (2092797, 344)


In [27]:
all_features = pd.read_parquet(location_features_node_level)

In [28]:
constants = []
for column in all_features.columns:
    if all_features[column].nunique(dropna=True) <= 1:
        del all_features[column]
        constants.append(column)
print(f"Deleted {len(constants)} constant columns")

Deleted 14 constant columns


In [29]:
medians = {}
for column in all_features.columns:
    medians[column] = np.nanmedian(all_features[column])

In [30]:
delta = round(time.time() - start_script)
print(f"Script executed in {timedelta(seconds=delta)}")

Script executed in 1:05:06


In [31]:
def add_turnover_score(df_input):
    df_input.loc[:, "turnover_score"] = df_input["turnover"] / 100_000
    df_input.loc[df_input["turnover_score"] > 1, "turnover_score"] = 1
    return df_input

In [32]:
flows.loc[:, "amount"] = flows.apply(
    lambda x: x["source_amount"] * CURRENCY_RATES[x["source_currency"]], axis=1
)

flow_stats = []
for key, df in flows.groupby("id"):
    left = (
        df.loc[:, ["tgt", "amount"]]
        .rename(columns={"tgt": "src"})
        .groupby("src")
        .agg({"amount": "sum"})
    )
    right = df.groupby("src").agg({"amount": "sum"})
    result = left.join(right, how="outer", lsuffix="_left").fillna(0).reset_index()
    result.loc[:, "delta"] = result["amount_left"] - result["amount"]
    turnover = float(result[result["delta"] > 0]["delta"].sum())
    turnover_weight = (
        result.set_index("src").apply(
            lambda x: max([x["amount_left"], x["amount"]]), axis=1
        )
        / turnover
    ).to_dict()
    turnover_weight = {
        k: v / (sum(turnover_weight.values()) / len(turnover_weight)) for k, v in turnover_weight.items()
    }
    nodes = sorted(set(df["src"]).union(df["tgt"]))
    flow_stats.append(
        {"id": key, "nodes": nodes, "turnover": int(np.ceil(turnover)), "turnover_weight": turnover_weight}
    )
flow_stats = pd.DataFrame(flow_stats)
flow_stats = add_turnover_score(flow_stats)

In [33]:
%%time

print("Training the model")

anomalies = all_features.loc[:, ["turnover"]].fillna(0)
model = IsolationForest(n_estimators=10_000)
anomalies.loc[:, "anomaly_score"] = model.fit(
    all_features.fillna(medians)
).decision_function(all_features.fillna(medians))
anomalies = anomalies.sort_values("anomaly_score", ascending=True)
anomalies = add_turnover_score(anomalies)

Training the model
CPU times: user 6min 18s, sys: 31.1 s, total: 6min 49s
Wall time: 6min 53s


In [34]:
count_50_perc = round(anomalies.shape[0] * 0.5)
top_50_perc_normal = anomalies.sort_values("anomaly_score", ascending=False).head(count_50_perc)
top_50_perc_normal = set(top_50_perc_normal.index)
top_50_perc_anom = anomalies.sort_values("anomaly_score", ascending=True).head(count_50_perc)
top_50_perc_anom.to_parquet("top_50_perc_anom.parquet")

In [1]:
communities_2_hop_dict = dict(communities_2_hop)
communities_filtered = defaultdict(list)
for node, comm in communities_2_hop_dict.items():
    comm = tuple(sorted(comm - top_50_perc_normal))
    communities_filtered[comm].append(node)

In [ ]:
communities_filtered_rev = {}
for key, value in communities_filtered.items():
    for x in value:
        communities_filtered_rev[x] = key

In [ ]:
libra_ml_accounts = 600
libra_anomalies_counts = [385, 770, 1925, 3851]
libra_anomalies_actuals_ratio = [x / libra_ml_accounts for x in libra_anomalies_counts]
ibm_anomalies_counts = [round(flows["id"].nunique() * x) for x in libra_anomalies_actuals_ratio]
ibm_anomalies_counts

In [ ]:
%%time

results = []
for anomalies_count in ibm_anomalies_counts:
    processed = set()
    top_communities = {}
    for index, row in anomalies.iterrows():
        comm = set(communities_filtered_rev[index]) - processed
        if len(comm) > 2:
            processed = processed.union(comm)
            top_communities[index] = comm
        if len(top_communities) >= anomalies_count:
            break
    communities_shortlisted = {
        k: v for k, v in top_communities.items()
    }
    max_comm_size = max(
        [len(x) for x in communities_shortlisted.values()] + [len(x) for x in flow_stats["turnover_weight"]]
    ) + 1
    assert len(top_communities) == len(communities_shortlisted)
    tp, fp, tn, fn = cw_confusion_matrix(flow_stats, communities_shortlisted, max_comm_size, anomalies, flows_hash)
    recall, f1 = cw_recall(tp, fn), cw_f1(tp, fp, fn)
    print(f"{recall=} | {max_comm_size=} | {anomalies_count=}")
    anom_comm_sizes = [len(x) for x in communities_shortlisted.values()]
    results.append({
        "anomalies_count": anomalies_count,
        "recall": recall,
        "max_comm_size": max_comm_size,
        "mean_comm_size": np.mean(anom_comm_sizes),
        "median_comm_size": np.median(anom_comm_sizes),
        "confusion_matrix": [tp, fp, tn, fn],
    })
results = pd.DataFrame(results)
results.to_parquet(f".{os.sep}results{os.sep}{s.OUTPUT_POSTFIX[1:]}-no-reduction.parquet")

In [3]:
results = pd.read_parquet(f".{os.sep}results{os.sep}{s.OUTPUT_POSTFIX[1:]}-no-reduction.parquet")